In [1]:
#Basic Libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint
import sys
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
#Load Dataset
df=pd.read_csv('/home/smriti/Desktop/NLP/MITACS/Anxiety/Data/CSV/Academic/Acad2015.csv')
#getting rid of NaN
df=df.replace(np.nan, '', regex=True)
#Combining title and text
df["Text"] = df["Title"] + df["Abstract"]
#Now that we don't need Title or Text, we drop those columns before saving the file
df=df.drop(['Title', 'Abstract'], axis = 1)

/home/smriti/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [4]:
df=df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
df.head()

,StoreId,documentType,year,Text
0,1758380839,"Journal , Peer Reviewed Journal , Journal Art...",2015,Anxiety and depression during pregnancy in Cen...
1,1758182902,"Journal , Peer Reviewed Journal , Journal Art...",2015,Changes in mental health in compliers and non-...
2,1756079990,"Journal , Peer Reviewed Journal , Journal Art...",2015,Psychiatric and neurological disorders in late...
3,1756079575,"Journal , Peer Reviewed Journal , Journal Art...",2015,Enthusiasm for homework and improvement of psy...
4,1753450094,"Journal , Peer Reviewed Journal , Journal Art...",2015,Cost-effectiveness of blended vs. Face-to-face...


In [6]:
# Convert to list
data=df.Text.values.tolist()
# Remove new line characters
data=[re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data=[re.sub("\'", "", sent) for sent in data]

In [7]:
#Function to clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

In [8]:
#Clean up text
data_words=list(sent_to_words(data))
print(data_words[:1])

[['anxiety', 'and', 'depression', 'during', 'pregnancy', 'in', 'central', 'america', 'cross', 'sectional', 'study', 'among', 'pregnant', 'women', 'in', 'the', 'developing', 'country', 'around', 'the', 'world', 'maternal', 'psychopathology', 'during', 'pregnancy', 'is', 'associated', 'with', 'range', 'of', 'negative', 'consequences', 'for', 'mother', 'and', 'child', 'nevertheless', 'in', 'central', 'america', 'the', 'magnitude', 'of', 'this', 'public', 'health', 'problem', 'is', 'still', 'unknown', 'the', 'objective', 'of', 'this', 'first', 'explorative', 'study', 'was', 'to', 'investigate', 'the', 'prevalence', 'and', 'severity', 'of', 'anxiety', 'and', 'depression', 'during', 'pregnancy', 'in', 'the', 'central', 'american', 'developing', 'country', 'nicaragua', 'as', 'well', 'as', 'the', 'availability', 'of', 'mental', 'health', 'care', 'and', 'to', 'compare', 'with', 'developed', 'country', 'methods', 'population', 'based', 'cohort', 'of', 'pregnant', 'women', 'in', 'nicaragua', 'was

In [9]:
# Build the bigram and trigram models
bigram=gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram=gensim.models.Phrases(bigram[data_words], threshold=100)

In [10]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod=gensim.models.phrases.Phraser(trigram)

In [11]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['anxiety', 'and', 'depression', 'during', 'pregnancy', 'in', 'central', 'america', 'cross_sectional', 'study', 'among', 'pregnant', 'women', 'in', 'the', 'developing', 'country', 'around', 'the', 'world', 'maternal', 'psychopathology', 'during', 'pregnancy', 'is', 'associated', 'with', 'range', 'of', 'negative', 'consequences', 'for', 'mother', 'and', 'child', 'nevertheless', 'in', 'central', 'america', 'the', 'magnitude', 'of', 'this', 'public', 'health', 'problem', 'is', 'still', 'unknown', 'the', 'objective', 'of', 'this', 'first', 'explorative', 'study', 'was', 'to', 'investigate', 'the', 'prevalence', 'and', 'severity', 'of', 'anxiety', 'and', 'depression', 'during', 'pregnancy', 'in', 'the', 'central', 'american', 'developing', 'country', 'nicaragua', 'as', 'well', 'as', 'the', 'availability', 'of', 'mental_health', 'care', 'and', 'to', 'compare', 'with', 'developed', 'country', 'methods', 'population', 'based', 'cohort', 'of', 'pregnant', 'women', 'in', 'nicaragua', 'was', 'com

In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [13]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

# Create Dictionary needed for topic modelling
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=80,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

[['anxiety', 'pregnancy', 'study', 'pregnant', 'woman', 'develop', 'country', 'maternal', 'psychopathology', 'pregnancy', 'associate', 'mother', 'nevertheless', 'health', 'problem', 'still', 'unknown', 'objective', 'first', 'explorative', 'study', 'investigate', 'prevalence', 'severity', 'anxiety', 'depression', 'pregnancy', 'develop', 'country', 'availability', 'care', 'compare', 'develop', 'country', 'method', 'population', 'base', 'cohort', 'pregnant', 'woman', 'compare', 'symptom', 'anxiety', 'state', 'trait', 'anxiety', 'depression', 'depression', 'scale', 'association', 'woman', 'knowledge', 'reach', 'professional', 'psychological', 'support', 'assess', 'use', 'multivariable', 'linear', 'regression', 'analyse', 'result', 'nicaraguan', 'woman', 'symptom', 'anxiety', 'symptom', 'depression', 'dutch', 'woman', 'symptom', 'score', 'anxiety', 'depression', 'significantly', 'high', 'however', 'woman', 'indicate', 'professional', 'psychological', 'help', 'available', 'nicaraguan', 'preg

In [15]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(29,
  '0.000*"risk" + 0.000*"disorder" + 0.000*"analyse" + 0.000*"conscription" + '
  '0.000*"psychiatric" + 0.000*"conviction" + 0.000*"personality" + '
  '0.000*"factor" + 0.000*"neurological" + 0.000*"early"'),
 (17,
  '0.000*"patient" + 0.000*"depression" + 0.000*"antidepressant" + '
  '0.000*"moderate" + 0.000*"primary" + 0.000*"care" + 0.000*"major" + '
  '0.000*"treat" + 0.000*"study" + 0.000*"mild"'),
 (26,
  '0.000*"group" + 0.000*"control" + 0.000*"study" + 0.000*"cost_effective" + '
  '0.000*"psychotic" + 0.000*"appear" + 0.000*"therapy" + 0.000*"clinical" + '
  '0.000*"people" + 0.000*"disorder"'),
 (49,
  '0.000*"disorder" + 0.000*"eat" + 0.000*"treatment" + 0.000*"evidence" + '
  '0.000*"clinical" + 0.000*"therapy" + 0.000*"cbt" + 0.000*"patient" + '
  '0.000*"setting" + 0.000*"routine"'),
 (32,
  '0.000*"treatment" + 0.000*"cbt" + 0.000*"adolescent" + 0.000*"depression" + '
  '0.000*"strategy" + 0.000*"improvement" + 0.000*"significantly" + '
  '0.000*"intervention" + 

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -12.828547516936805


In [17]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.43074179611953534


In [18]:
num_topics = 80
topic_words = []
for i in range(num_topics):
    tt = lda_model.get_topic_terms(i,10)
    topic_words.append([id2word[pair[0]] for pair in tt])

In [19]:
for i in range(0,80):
    print(topic_words[i])

['intervention', 'disorder', 'anxiety', 'woman', 'cocaine', 'concentration', 'depression', 'psychiatric', 'risk', 'increase']
['risk', 'disorder', 'history', 'psychiatric', 'age', 'study', 'increase', 'individual', 'relative', 'psychosis']
['role', 'point', 'emerge', 'anxiety', 'model', 'vulnerability', 'individual', 'dysregulation', 'investigate', 'tendency']
['trauma', 'event', 'negative', 'worry', 'mediate', 'predict', 'ptsd', 'reaction', 'traumatic', 'rumination']
['stress', 'response', 'environmental', 'cancer', 'adolescent', 'function', 'boy', 'several', 'lead', 'pituitary']
['avoidance', 'frequent', 'theory', 'understand', 'behavior', 'impact', 'chronic', 'poorly', 'way', 'research']
['child', 'parent', 'tool', 'diagnostic', 'cut', 'screening', 'score', 'pediatric', 'clinical', 'sensitivity']
['anxiety', 'disorder', 'child', 'symptom', 'age', 'study', 'year', 'depression', 'depressive', 'childhood']
['session', 'depressed', 'enhance', 'best', 'homework', 'total', 'patient', 'abl

In [20]:
df=pd.DataFrame({'Year':[],'Source':[],'Topic_ID':[],'Most_freq_words':[]})
df.head()

,Most_freq_words,Source,Topic_ID,Year


In [21]:
df['Most_freq_words']=topic_words

In [22]:
df = df.assign(Year='2015')
df = df.assign(Source='Academic')
df.shape

(80, 4)

In [23]:
ls=[]
for i in range(0,80):
    ls.append(i)
df['Topic_ID']=ls

In [24]:
df.head()

,Most_freq_words,Source,Topic_ID,Year
0,"[intervention, disorder, anxiety, woman, cocai...",Academic,0,2015
1,"[risk, disorder, history, psychiatric, age, st...",Academic,1,2015
2,"[role, point, emerge, anxiety, model, vulnerab...",Academic,2,2015
3,"[trauma, event, negative, worry, mediate, pred...",Academic,3,2015
4,"[stress, response, environmental, cancer, adol...",Academic,4,2015


In [25]:
df.to_csv("topic_words_j2015.csv")